In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import os
import shutil
import pandas as pd
from skimage.morphology import skeletonize
from shapely.geometry import LineString

def load_calibration_data(csv_path):
    return pd.read_csv(csv_path)

def move_image_to_folder(parent_folder, image_file):
    image_name_no_ext = os.path.splitext(image_file)[0]
    image_path = os.path.join(parent_folder, image_file)
    image_folder = os.path.join(parent_folder, image_name_no_ext)
    os.makedirs(image_folder, exist_ok=True)
    new_image_path = os.path.join(image_folder, image_file)
    shutil.move(image_path, new_image_path)
    return new_image_path, image_folder, image_name_no_ext

def get_mm_to_pixel_ratio(calibration_data, image_name_no_ext):
    row = calibration_data[calibration_data["file_name"] == image_name_no_ext]
    if not row.empty:
        return row.iloc[0]["distance"] / row.iloc[0]["pixel"]
    print(f"Skipping {image_name_no_ext}: Calibration data not found.")
    return None

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    shifted = cv2.pyrMeanShiftFiltering(image, sp=20, sr=40)
    gray = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
    return image, cleaned, gray


def build_graph_from_skeleton(skeleton):

    G = nx.Graph()
    h, w = skeleton.shape

    points = np.column_stack(np.where(skeleton > 0)) 
    if len(points) == 0:
        return G  

    for y, x in points:
        G.add_node((y, x))

    for y, x in points:
        neighbors = [(y + dy, x + dx) for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
                 if (0 <= y + dy < skeleton.shape[0] and 0 <= x + dx < skeleton.shape[1] and skeleton[y + dy, x + dx] > 0)]
        for neighbor_y, neighbor_x in neighbors:
            distance = np.sqrt((y - neighbor_y) ** 2 + (x - neighbor_x) ** 2)  
            G.add_edge((y, x), (neighbor_y, neighbor_x), weight=distance)  
    return G

def simplify_path(path, tolerance=2.0):

    if len(path) < 3: 
        return path
    line = LineString(path)
    simplified = line.simplify(tolerance, preserve_topology=False)
    return list(simplified.coords)

def reconnect_path(G, path):

    new_G = G.copy()  
    total_length = 0

    for i in range(len(path) - 1):
        node1 = tuple(map(int, path[i]))  
        node2 = tuple(map(int, path[i + 1]))

        if node1 not in new_G:
            new_G.add_node(node1)
        if node2 not in new_G:
            new_G.add_node(node2)

        if node2 not in new_G[node1]:
            distance = ((node1[0] - node2[0])**2 + (node1[1] - node2[1])**2) ** 0.5
            new_G.add_edge(node1, node2, weight=distance)

        total_length += new_G[node1][node2]['weight']

    return total_length

def find_farthest_nodes(G):

    if len(G) == 0:
        return None, None

    start_node = list(G.nodes)[0]  
    lengths = nx.single_source_dijkstra_path_length(G, start_node, weight='weight')
    farthest_node_1 = max(lengths, key=lengths.get)

    lengths = nx.single_source_dijkstra_path_length(G, farthest_node_1, weight='weight')
    farthest_node_2 = max(lengths, key=lengths.get)

    return farthest_node_1, farthest_node_2

def find_shortest_path(G, node1, node2):

    if node1 is None or node2 is None:
        return [], 0

    path = nx.shortest_path(G, source=node1, target=node2, weight='weight')
    return path

def calculate_path_distance(G, path):

    total_length = 0
    for i in range(len(path) - 1):
        total_length += G[path[i]][path[i + 1]]['weight'] 
    return total_length

def analyze_sprouts(image, cleaned, gray, mm_to_pixel_ratio):
    contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    output_image = image.copy()
    skeleton_image = np.zeros_like(gray)
    sprout_data = []
    used_label_positions = [] 

    for idx, contour in enumerate(contours):
        if cv2.contourArea(contour) < 300:
            continue

        cv2.drawContours(output_image, [contour], -1, (255, 0, 0), 1)

        mask = np.zeros_like(gray, dtype=np.uint8)
        cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)
        skeleton = skeletonize(mask > 0).astype(np.uint8) * 255

        skeleton_image = cv2.bitwise_or(skeleton_image, skeleton)

        G = build_graph_from_skeleton(skeleton)

        if len(G.nodes) == 0:
            continue

        node1, node2 = find_farthest_nodes(G)

        shortest_path = find_shortest_path(G, node1, node2)

        simplified_path = simplify_path(shortest_path, tolerance=2.0)

        path_length_pixels = reconnect_path(G, simplified_path)
        path_length_mm = path_length_pixels * mm_to_pixel_ratio
        sprout_data.append([idx + 1, path_length_pixels, path_length_mm])

        if node1 and node2:
            cv2.circle(output_image, (node1[1], node1[0]), 5, (0, 255, 0), -1)  
            cv2.circle(output_image, (node2[1], node2[0]), 5, (0, 255, 0), -1)  

        for i in range(len(simplified_path) - 1):
            pt1 = (int(simplified_path[i][1]), int(simplified_path[i][0]))
            pt2 = (int(simplified_path[i + 1][1]), int(simplified_path[i + 1][0]))
            cv2.line(output_image, pt1, pt2, (0, 0, 255), 1)  

        label_text = f"{idx + 1}: {path_length_mm:.2f} mm"
        label_position = (node2[1] + 10, node2[0] - 10)  

        while any(np.linalg.norm(np.array(label_position) - np.array(pos)) < 20 for pos in used_label_positions):
            label_position = (label_position[0], label_position[1] + 15)  

        used_label_positions.append(label_position)  
        cv2.putText(output_image, label_text, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    return output_image, skeleton_image, sprout_data

def save_results(image_folder, image_name_no_ext, output_image, skeleton_image, sprout_data):
    cv2.imwrite(os.path.join(image_folder, f"skeletons_{image_name_no_ext}.jpg"), skeleton_image)
    cv2.imwrite(os.path.join(image_folder, f"length_measurement_{image_name_no_ext}.jpg"), output_image)
    pd.DataFrame(sprout_data, columns=["Sprout Number", "Pixels", "Millimeters"]).to_csv(
        os.path.join(image_folder, f"sprout_lengths_{image_name_no_ext}.csv"), index=False
    )

def main():
    parent_folder = "test_run"
    csv_path = os.path.join(parent_folder, "batch3.csv")
    calibration_data = load_calibration_data(csv_path)
    image_files = [f for f in os.listdir(parent_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    
    for image_file in image_files:
        new_image_path, image_folder, image_name_no_ext = move_image_to_folder(parent_folder, image_file)
        mm_to_pixel_ratio = get_mm_to_pixel_ratio(calibration_data, image_name_no_ext)
        
        if mm_to_pixel_ratio is None:
            continue
        
        image, cleaned, gray = preprocess_image(new_image_path)
        output_image, skeleton_image, sprout_data = analyze_sprouts(image, cleaned, gray, mm_to_pixel_ratio)
        save_results(image_folder, image_name_no_ext, output_image, skeleton_image, sprout_data)
        print(f"Processed {image_file}. Results saved in {image_folder}.")

if __name__ == "__main__":
    main()